In [13]:
import pandas as pd

# Load dataset
df = pd.read_csv('/content/generated_inventory_dataset.csv')

# Display the first few rows of the dataset
print(df.head())


  RegionName                  CountryName          State           City  \
0   Missouri                      Bahamas    Mississippi  Martinezshire   
1  Wisconsin                        Kenya   South Dakota   Barajasburgh   
2   Nebraska  French Southern Territories       Delaware    Johnchester   
3    Arizona                     Cambodia        Florida    Anthonytown   
4   Kentucky                        Samoa  Massachusetts   Charlesmouth   

   PostalCode                                   WarehouseAddress  \
0       50167  6818 Robinson Greens Apt. 594\nJesusport, MA 6...   
1        8169  6711 Gonzalez Run Apt. 345\nLucaschester, IL 7...   
2       82563  39749 Marisa Brooks Apt. 138\nSantosmouth, NY ...   
3       47333  770 Daniels Plaza Suite 597\nLake Amber, SC 06983   
4       28711           93041 Rogers Mall\nDouglasfurt, NH 17390   

                  WarehouseName      EmployeeName               EmployeeEmail  \
0                     Davis Inc      Steven Perez    youngj

In [30]:
import heapq

class Graph:
    def __init__(self):
        self.nodes = set()
        self.edges = {}
        self.distances = {}

    def add_node(self, value):
        self.nodes.add(value)
        self.edges[value] = []

    def add_edge(self, from_node, to_node, distance):
        self.edges[from_node].append(to_node)
        self.distances[(from_node, to_node)] = distance

def dijkstra(graph, initial):
    visited = {initial: 0}
    path = {}

    nodes = set(graph.nodes)

    while nodes:
        min_node = None
        for node in nodes:
            if node in visited:
                if min_node is None:
                    min_node = node
                elif visited[node] < visited[min_node]:
                    min_node = node

        if min_node is None:
            break

        nodes.remove(min_node)
        current_weight = visited[min_node]

        for edge in graph.edges[min_node]:
            weight = current_weight + graph.distances[(min_node, edge)]
            if edge not in visited or weight < visited[edge]:
                visited[edge] = weight
                path[edge] = min_node

    return visited, path

# Create graph from dataset
graph = Graph()
for node in pd.concat([df['Origin_Location'], df['Destination_Location']]).unique():
    graph.add_node(node)

# Assuming df has columns 'Origin_Location', 'Destination_Location', and 'TRANSPORTATION_DISTANCE_IN_KM'
for _, row in df.iterrows():
    graph.add_edge(row['Origin_Location'], row['Destination_Location'], row['TRANSPORTATION_DISTANCE_IN_KM'])

start_location = 'North Lisaton'  # Replace with actual starting location
distances, paths = dijkstra(graph, start_location)
print(distances)


{'North Lisaton': 0, 'Derrickton': 1586.94}


In [31]:
import random
import pandas as pd

# Load dataset
df = pd.read_csv('/content/generated_inventory_dataset.csv')

class GeneticAlgorithm:
    def __init__(self, population_size, mutation_rate, generations):
        self.population_size = population_size
        self.mutation_rate = mutation_rate
        self.generations = generations

    def initialize_population(self):
        population = []
        for _ in range(self.population_size):
            individual = self.create_individual()
            population.append(individual)
        return population

    def create_individual(self):
        # Create a random individual (solution)
        # Example: A list of route indices
        individual = df.sample(frac=1).index.tolist()
        return individual

    def fitness(self, individual):
        # Fitness function to evaluate the individual
        # Example: Total distance of the route
        total_distance = sum(df.loc[individual, 'TRANSPORTATION_DISTANCE_IN_KM'])
        return 1 / total_distance

    def selection(self, population):
        # Select individuals based on fitness (roulette wheel selection)
        fitnesses = [self.fitness(ind) for ind in population]
        total_fitness = sum(fitnesses)
        probs = [fit / total_fitness for fit in fitnesses]
        selected = random.choices(population, weights=probs, k=2)
        return selected

    def crossover(self, parent1, parent2):
        # Single point crossover
        point = random.randint(1, len(parent1) - 1)
        child1 = parent1[:point] + parent2[point:]
        child2 = parent2[:point] + parent1[point:]
        return child1, child2

    def mutate(self, individual):
        # Randomly swap two elements in the individual
        if random.random() < self.mutation_rate:
            idx1, idx2 = random.sample(range(len(individual)), 2)
            individual[idx1], individual[idx2] = individual[idx2], individual[idx1]

    def run(self):
        population = self.initialize_population()
        for generation in range(self.generations):
            new_population = []
            for _ in range(self.population_size // 2):
                parent1, parent2 = self.selection(population)
                child1, child2 = self.crossover(parent1, parent2)
                self.mutate(child1)
                self.mutate(child2)
                new_population.extend([child1, child2])
            population = new_population
            best_individual = max(population, key=self.fitness)
            print(f'Generation {generation}: Best fitness = {self.fitness(best_individual)}')
        return best_individual

# Example usage
ga = GeneticAlgorithm(population_size=50, mutation_rate=0.01, generations=10)
best_route = ga.run()
print(f'Optimized Route: {best_route}')


Generation 0: Best fitness = 9.948588777261633e-07
Generation 1: Best fitness = 9.98712918713137e-07
Generation 2: Best fitness = 1.0007882408342471e-06
Generation 3: Best fitness = 1.0122293911006983e-06
Generation 4: Best fitness = 9.984233497514736e-07
Generation 5: Best fitness = 9.976779643985408e-07
Generation 6: Best fitness = 1.0136378690728975e-06
Generation 7: Best fitness = 1.02602243237989e-06
Generation 8: Best fitness = 1.0033794522316785e-06
Generation 9: Best fitness = 1.0080116971290155e-06
Optimized Route: [240, 317, 751, 866, 453, 325, 808, 196, 519, 12, 171, 550, 804, 95, 298, 592, 52, 518, 241, 112, 249, 598, 998, 15, 734, 150, 923, 589, 51, 357, 43, 981, 835, 899, 969, 908, 223, 163, 392, 405, 175, 695, 796, 892, 495, 452, 265, 768, 78, 158, 635, 659, 109, 802, 658, 962, 174, 525, 377, 415, 836, 204, 932, 586, 188, 73, 564, 212, 521, 720, 341, 745, 468, 29, 20, 473, 561, 338, 557, 988, 474, 538, 460, 813, 597, 334, 648, 526, 611, 425, 123, 556, 523, 877, 773, 40, 

In [34]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
import pandas as pd

# Load dataset
df = pd.read_csv('/content/generated_inventory_dataset.csv')

# One-hot encode categorical features
X = pd.get_dummies(df[['Origin_Location', 'Destination_Location']])
y = df['TRANSPORTATION_DISTANCE_IN_KM']

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize and train the model
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Make predictions
predictions = model.predict(X_test)

# Evaluate the model
mae = mean_absolute_error(y_test, predictions)
print(f'Mean Absolute Error: {mae}')


Mean Absolute Error: 483.41889900000007


In [36]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

# Assume df has additional columns relevant for workforce and inventory management

# One-hot encode categorical features
X = pd.get_dummies(df[['Origin_Location', 'Destination_Location']])
y = df['TRANSPORTATION_DISTANCE_IN_KM']

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize and train the model
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Make predictions
predictions = model.predict(X_test)

# Evaluate the model
mae = mean_absolute_error(y_test, predictions)
print(f'Mean Absolute Error: {mae}')

# Function to recommend inventory replenishment
def recommend_replenishment(inventory_df):
    low_stock_items = inventory_df[inventory_df['Stock_Level'] < 20]
    print("Items to be replenished:")
    print(low_stock_items)

recommend_replenishment(inventory_df)

# Function to assign tasks to available workers
def assign_tasks(workforce_df):
    available_workers = workforce_df[workforce_df['Status'] == 'Available']
    tasks = []
    for i in range(len(available_workers)):
        task = {
            'Worker_ID': available_workers.iloc[i]['Worker_ID'],
            'Task': f"Deliver items from {available_workers.iloc[i]['Current_Location']} to a nearby destination"
        }
        tasks.append(task)
    return tasks

tasks = assign_tasks(workforce_df)
print("Assigned Tasks:")
for task in tasks:
    print(task)


Mean Absolute Error: 483.41889900000007
Items to be replenished:
    Item_ID            Location  Stock_Level
14       15        West Jillian            2
15       16         West Regina            0
16       17     Port Scottville            4
18       19           Ayalaberg           13
20       21         Steelehaven            8
22       23       Fitzgeraldton           14
31       32       New Chadburgh            3
34       35       South Michael           14
38       39           Karenside           12
49       50       Martinezburgh            5
61       62         Port Nicole            0
65       66          West Susan            2
69       70    East Christopher            8
85       86          Andrewstad           11
87       88         Port Leslie            1
88       89     North Heidibury            2
92       93  East Justinchester            1
93       94       North Derrick            1
99      100         Stokesmouth            0
Assigned Tasks:
{'Worker_ID': 1, 'T

In [40]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

# Load dataset
df = pd.read_csv('/content/generated_inventory_dataset.csv')

# Simulate workforce data
np.random.seed(42)
workforce_data = {
    'Worker_ID': range(1, 21),  # 20 workers
    'Current_Location': np.random.choice(df['Origin_Location'].unique(), 20),
    'Status': np.random.choice(['Available', 'Busy'], 20, p=[0.7, 0.3])
}
workforce_df = pd.DataFrame(workforce_data)

# Simulate inventory data
inventory_data = {
    'Item_ID': range(1, 101),  # 100 items
    'Location': np.random.choice(df['Origin_Location'].unique(), 100),
    'Stock_Level': np.random.randint(0, 100, 100)
}
inventory_df = pd.DataFrame(inventory_data)

# Display sample data
print("Workforce Data:")
print(workforce_df.head())
print("\nInventory Data:")
print(inventory_df.head())

# Predictive Modeling for Transportation Distances

# One-hot encode categorical features
X = pd.get_dummies(df[['Origin_Location', 'Destination_Location']])
y = df['TRANSPORTATION_DISTANCE_IN_KM']

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize and train the model
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Make predictions
predictions = model.predict(X_test)

# Evaluate the model
mae = mean_absolute_error(y_test, predictions)
print(f'\nMean Absolute Error: {mae}')

# Function to recommend inventory replenishment
def recommend_replenishment(inventory_df):
    low_stock_items = inventory_df[inventory_df['Stock_Level'] < 20]
    print("\nItems to be replenished:")
    print(low_stock_items)

# Function to assign tasks to available workers
def assign_tasks(workforce_df):
    available_workers = workforce_df[workforce_df['Status'] == 'Available']
    tasks = []
    for i in range(len(available_workers)):
        task = {
            'Worker_ID': available_workers.iloc[i]['Worker_ID'],
            'Task': f"Deliver items from {available_workers.iloc[i]['Current_Location']} to a nearby destination"
        }
        tasks.append(task)
    return tasks

# Recommend replenishment
recommend_replenishment(inventory_df)

# Assign tasks
tasks = assign_tasks(workforce_df)
print("\nAssigned Tasks:")
for task in tasks:
    print(task)


Workforce Data:
   Worker_ID      Current_Location     Status
0          1  South Melindaborough  Available
1          2             Ewingfort       Busy
2          3       Port Scottshire       Busy
3          4           Michealland  Available
4          5             Longshire  Available

Inventory Data:
   Item_ID     Location  Stock_Level
0        1    Mcgeefort           47
1        2  Danielshire           22
2        3   Gravesstad           61
3        4     Johnbury           87
4        5  Danielburgh           36

Mean Absolute Error: 483.41889900000007

Items to be replenished:
    Item_ID            Location  Stock_Level
14       15        West Jillian            2
15       16         West Regina            0
16       17     Port Scottville            4
18       19           Ayalaberg           13
20       21         Steelehaven            8
22       23       Fitzgeraldton           14
31       32       New Chadburgh            3
34       35       South Michael           

In [41]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

# Load dataset
df = pd.read_csv('/content/generated_inventory_dataset.csv')

# Simulate workforce data
np.random.seed(42)
workforce_data = {
    'Worker_ID': range(1, 101),  # 100 workers
    'Current_Location': np.random.choice(df['Origin_Location'].unique(), 100),
    'Status': np.random.choice(['Available', 'Busy'], 100, p=[0.7, 0.3])
}
workforce_df = pd.DataFrame(workforce_data)

# Simulate inventory data
inventory_data = {
    'Item_ID': range(1, 101),  # 100 items
    'Location': np.random.choice(df['Origin_Location'].unique(), 100),
    'Stock_Level': np.random.randint(0, 100, 100)
}
inventory_df = pd.DataFrame(inventory_data)

# Display sample data
print("Workforce Data:")
print(workforce_df.head())
print("\nInventory Data:")
print(inventory_df.head())

# Predictive Modeling for Transportation Distances

# One-hot encode categorical features
X = pd.get_dummies(df[['Origin_Location', 'Destination_Location']])
y = df['TRANSPORTATION_DISTANCE_IN_KM']

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize and train the model
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Make predictions
predictions = model.predict(X_test)

# Evaluate the model
mae = mean_absolute_error(y_test, predictions)
print(f'\nMean Absolute Error: {mae}')

# Function to recommend inventory replenishment
def recommend_replenishment(inventory_df):
    low_stock_items = inventory_df[inventory_df['Stock_Level'] < 20]
    print("\nItems to be replenished:")
    print(low_stock_items)

# Function to assign tasks to available workers
def assign_tasks(workforce_df):
    available_workers = workforce_df[workforce_df['Status'] == 'Available']
    tasks = []
    for i in range(len(available_workers)):
        task = {
            'Worker_ID': available_workers.iloc[i]['Worker_ID'],
            'Task': f"Deliver items from {available_workers.iloc[i]['Current_Location']} to a nearby destination"
        }
        tasks.append(task)
    return tasks

# Recommend replenishment
recommend_replenishment(inventory_df)

# Assign tasks
tasks = assign_tasks(workforce_df)
print("\nAssigned Tasks:")
for task in tasks:
    print(task)


Workforce Data:
   Worker_ID      Current_Location     Status
0          1  South Melindaborough       Busy
1          2             Ewingfort       Busy
2          3       Port Scottshire  Available
3          4           Michealland       Busy
4          5             Longshire  Available

Inventory Data:
   Item_ID     Location  Stock_Level
0        1   Danielfort           88
1        2    New Anita           98
2        3  Brookeburgh           24
3        4  Michaelland           92
4        5    New Susan           17

Mean Absolute Error: 483.41889900000007

Items to be replenished:
    Item_ID           Location  Stock_Level
4         5          New Susan           17
16       17  Port Gabrielshire           13
19       20    North Tammybury           19
20       21  West Gabriellaton            7
21       22        Parkershire            6
23       24        Johnsonview           16
34       35      South William            7
47       48        Lake Julian           18
48    

In [42]:
def allocate_tasks(employees, tasks):
    allocation = []
    for task in tasks.iterrows():
        for employee in employees.iterrows():
            if task[1]['CategoryName'] == employee[1]['EmployeeJobTitle'] and employee[1]['EmployeeEmail'] >= task[1]['actual_eta']:
                allocation.append({
                    'BookingID': task[1]['BookingID'],
                    'EmployeeName': employee[1]['EmployeeName']
                })
                employees.loc[employee[0], 'availability'] -= task[1]['estimated_time']
                break
    return allocation

# Example usage with generated data
employees = df[['EmployeeName', 'EmployeeJobTitle', 'EmployeeHireDate']].drop_duplicates()
tasks = df[['BookingID', 'CategoryName', 'ProductName']].drop_duplicates()

task_allocation = allocate_tasks(employees, tasks)
for allocation in task_allocation:
    print(f"Task {allocation['BookingID']} assigned to Employee {allocation['EmployeeName']}")

In [45]:
import pandas as pd
import numpy as np

# Load dataset
df = pd.read_csv('/content/generated_inventory_dataset.csv')

# Assuming df has the necessary columns
employees = df[['EmployeeName', 'EmployeeJobTitle', 'EmployeeHireDate', 'EmployeeEmail']].drop_duplicates()
tasks = df[['BookingID', 'CategoryName', 'ProductName', 'actual_eta', 'Planned_ETA']].drop_duplicates()

# Initialize availability for employees (assuming full availability initially, e.g., 8 hours)
employees['availability'] = 8 * 60  # in minutes

def allocate_tasks(employees, tasks):
    allocation = []
    for _, task in tasks.iterrows():
        for _, employee in employees.iterrows():
            if task['CategoryName'] == employee['EmployeeJobTitle'] and employee['availability'] >= task['Planned_ETA']:
                allocation.append({
                    'BookingID': task['BookingID'],
                    'EmployeeName': employee['EmployeeName']
                })
                employees.loc[employees['EmployeeName'] == employee['EmployeeName'], 'availability'] -= task['Planned_ETA']
                break
    return allocation

# Example usage with generated data
task_allocation = allocate_tasks(employees, tasks)

for allocation in task_allocation:
    print(f"Task {allocation['BookingID']} assigned to Employee {allocation['EmployeeName']}")


In [48]:
import pandas as pd
import numpy as np

# Load dataset
df = pd.read_csv('/content/generated_inventory_dataset.csv')

# Assuming df has the necessary columns
employees = df[['EmployeeName', 'EmployeeJobTitle', 'EmployeeHireDate', 'EmployeeEmail']].drop_duplicates()
tasks = df[['BookingID', 'CategoryName', 'ProductName', 'actual_eta', 'Planned_ETA']].drop_duplicates()

# Initialize availability for employees (assuming full availability initially, e.g., 8 hours)
employees['availability'] = 8 * 60  # in minutes

def allocate_tasks(employees, tasks):
    allocation = []
    for _, task in tasks.iterrows():
        print(f"Trying to allocate Task {task['BookingID']} with estimated time {task['actual_eta']}")
        for _, employee in employees.iterrows():
            if task['CategoryName'] == employee['EmployeeJobTitle']:
                if employee['availability'] >= task['actual_eta']:
                    allocation.append({
                        'BookingID': task['BookingID'],
                        'EmployeeName': employee['EmployeeName']
                    })
                    employees.loc[employees['EmployeeName'] == employee['EmployeeName'], 'availability'] -= task['actual_eta']
                    print(f"Task {task['BookingID']} assigned to Employee {employee['EmployeeName']}")
                    break
                else:
                    print(f"Employee {employee['EmployeeName']} does not have enough availability.")
        else:
            print(f"No suitable employee found for Task {task['BookingID']}.")
    return allocation

# Example usage with generated data
task_allocation = allocate_tasks(employees, tasks)

for allocation in task_allocation:
    print(f"Task {allocation['BookingID']} assigned to Employee {allocation['EmployeeName']}")


Trying to allocate Task adf76fae-be3e-4a57-b8aa-d91f47c00e9c with estimated time 2024-07-18 22:51:00.379937
No suitable employee found for Task adf76fae-be3e-4a57-b8aa-d91f47c00e9c.
Trying to allocate Task 0e6a2a85-222f-4039-85c5-f4f0f82497c3 with estimated time 2024-02-19 22:33:47.140664
No suitable employee found for Task 0e6a2a85-222f-4039-85c5-f4f0f82497c3.
Trying to allocate Task 40001b84-70b9-4c61-b3d5-19b360b28e28 with estimated time 2024-06-20 22:01:13.308612
No suitable employee found for Task 40001b84-70b9-4c61-b3d5-19b360b28e28.
Trying to allocate Task b3b3967a-0953-4ca8-9ad4-83c728c7c947 with estimated time 2024-02-26 13:04:37.522025
No suitable employee found for Task b3b3967a-0953-4ca8-9ad4-83c728c7c947.
Trying to allocate Task a6b83b9d-2cd1-4312-8b88-668351896865 with estimated time 2024-02-29 06:04:32.332506
No suitable employee found for Task a6b83b9d-2cd1-4312-8b88-668351896865.
Trying to allocate Task e4570e98-73c0-489a-84fa-d2488a0829b3 with estimated time 2024-01-0